In [2]:
# If notebook is run on google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [60]:
path = "/Users/kyradresen/Desktop/UvA Master Data Science /AFCS/Final project/"

calendar_df = pd.read_csv(path+'calendar_afcs2023.csv')
sell_prices_df = pd.read_csv(path+'sell_prices_afcs2023.csv')
sales_test = pd.read_csv(path+'/sales_test_validation_afcs2023.csv')
sales_train = pd.read_csv(path+'sales_train_validation_afcs2023.csv')
sample_submission = pd.read_csv(path+'sell_prices_afcs2023.csv')

In [61]:
date_range = pd.Series(pd.date_range(start="2011-01-29", end="2016-06-19", freq='D')).dt.strftime('%m/%d/%Y').to_list()
date_range.insert(0, "id")

In [62]:
sales_train.columns = date_range[:len(sales_train.columns)]
sales_train[['item_id', "Loc1", "Loc2", "Type"]] = sales_train.id.str.rsplit("_", n=3, expand=True)
sales_train = sales_train.drop("id", axis=1)
sales_train = sales_train.melt(id_vars=['item_id', "Loc1", "Loc2", "Type"], var_name="date", value_name="sales_amount")
sales_train["date"] = pd.to_datetime(sales_train["date"])
sales_train.head()

,item_id,Loc1,Loc2,Type,date,sales_amount
0,FOODS_3_001,TX,3,validation,2011-01-29,0
1,FOODS_3_002,TX,3,validation,2011-01-29,0
2,FOODS_3_003,TX,3,validation,2011-01-29,0
3,FOODS_3_004,TX,3,validation,2011-01-29,0
4,FOODS_3_005,TX,3,validation,2011-01-29,0


In [63]:
sales_train['date'] = pd.to_datetime(sales_train['date'])
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

# Merge the DataFrames on the 'date' column
sales_train = pd.merge(sales_train, calendar_df, on='date')

sales_train.head()

,item_id,Loc1,Loc2,Type,date,sales_amount,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX
0,FOODS_3_001,TX,3,validation,2011-01-29,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0
1,FOODS_3_002,TX,3,validation,2011-01-29,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0
2,FOODS_3_003,TX,3,validation,2011-01-29,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0
3,FOODS_3_004,TX,3,validation,2011-01-29,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0
4,FOODS_3_005,TX,3,validation,2011-01-29,0,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0


In [64]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1574399 entries, 0 to 1574398
Data columns (total 16 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   item_id       1574399 non-null  object        
 1   Loc1          1574399 non-null  object        
 2   Loc2          1574399 non-null  object        
 3   Type          1574399 non-null  object        
 4   date          1574399 non-null  datetime64[ns]
 5   sales_amount  1574399 non-null  int64         
 6   wm_yr_wk      1574399 non-null  int64         
 7   weekday       1574399 non-null  object        
 8   wday          1574399 non-null  int64         
 9   month         1574399 non-null  int64         
 10  year          1574399 non-null  int64         
 11  event_name_1  126742 non-null   object        
 12  event_type_1  126742 non-null   object        
 13  event_name_2  3292 non-null     object        
 14  event_type_2  3292 non-null     object        
 15

In [65]:
sales_train = sales_train.drop(columns=['Loc1', 'Loc2'])

In [66]:
f

,store_id,item_id,wm_yr_wk,sell_price
0,TX_3,FOODS_3_001,11101,2.28
1,TX_3,FOODS_3_001,11102,2.28
2,TX_3,FOODS_3_001,11103,2.28
3,TX_3,FOODS_3_001,11104,2.28
4,TX_3,FOODS_3_001,11105,2.28
...,...,...,...,...
185255,TX_3,FOODS_3_827,11617,1.00
185256,TX_3,FOODS_3_827,11618,1.00
185257,TX_3,FOODS_3_827,11619,1.00
185258,TX_3,FOODS_3_827,11620,1.00


In [68]:
# Merge the DataFrames on the 'week' and 'item_id' column
# left join because sell_prices_df misses rows where a product was not sold in a week
sales_train = pd.merge(sales_train, sell_prices_df, on=['wm_yr_wk', 'item_id'],how ='left')
sales_train = sales_train.drop('store_id',axis=1)
sales_train

KeyError: "['store_id'] not found in axis"

In [56]:
# duplicate column, we only need one week day column
sales_train=sales_train.drop('wday',axis=1)

In [57]:
sales_train

,item_id,Type,date,sales_amount,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX,sell_price
0,FOODS_3_001,validation,2011-01-29,0,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,2.28
1,FOODS_3_002,validation,2011-01-29,0,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,NaN
2,FOODS_3_003,validation,2011-01-29,0,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,NaN
3,FOODS_3_004,validation,2011-01-29,0,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,NaN
4,FOODS_3_005,validation,2011-01-29,0,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,1.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574394,FOODS_3_823,validation,2016-04-24,2,11613,Sunday,4,2016,NaN,NaN,NaN,NaN,0,2.98
1574395,FOODS_3_824,validation,2016-04-24,0,11613,Sunday,4,2016,NaN,NaN,NaN,NaN,0,2.48
1574396,FOODS_3_825,validation,2016-04-24,0,11613,Sunday,4,2016,NaN,NaN,NaN,NaN,0,3.98
1574397,FOODS_3_826,validation,2016-04-24,2,11613,Sunday,4,2016,NaN,NaN,NaN,NaN,0,1.28


In [46]:
# Now we need to do some feature engineering, think we need to:

# 1. create lagged & forward looking event variables, as events do not only have impact on the day itself.
# e.g. with christmas, people buy a lot before christmas, on christmas the store might even be closed, 
# and after christmas they maybe buy less than normal

# 2. I think we need to do something (normalize) with the sell_price. absolute price is not necessarily most relevant. Think we
# need to compare it with price of last week, as this indicates that there is a promotion or something like that. 
# Maybe it is easier if we do this before we merge the sell price table with the sales table

# 3. I think we also need to create lagged & forward looking (relative) price variables. Because a promotion this
# week could lead to less sales next week. And a promotion next week could also lead to customers anticipitating on
# this and waiting with certain purchases until next week. Maybe we should also create a dummy variable 'promotion'
# which equals 1 if the price of a product is lower than in previous week(s)

# 4. For weeks where there was no sale, we don;t have a price. So we need to fill in a price, e.g., by taking the
# price of last week. Or by taking the max (or average) price of past x weeks to avoid taking a price of a previous
# week where there was a promotion.

# 5. For days/weeks where there was no sale of a certain product, it could be because no one wanted to buy it, but
# it could also be because the product was out of stock. Maybe we should try to model out of stock. E.g., if 
# a product was not sold multiple days/weeks in a row, while normally it is sold quite often, we label it as out of.
# But this might be too complicated. In any case I think we can mention it in the report


In [ ]:
########################### Vars
#################################################################################
TARGET = 'sales'         # Our main target
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns